In [15]:
import numpy as np
from tkinter import *
import tkinter.scrolledtext as tkst
from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.animation import FuncAnimation
import math
import pandas as pd

marathon_2015_2017 = pd.read_csv("./marathon_2015_2017.csv")
marathon_2015_2017.head(2)

,Name,Age,M/F,City,State,Country,5K,10K,15K,20K,...,25K,30K,35K,40K,Pace,Official Time,Overall,Gender,Division,Year
0,"Desisa, Lelisa",25,M,Ambo,NaN,ETH,883,1783,2697,3629,...,4567,5520,6479,7359,296,7757,1,1,1,2015
1,"Tsegay, Yemane Adhane",30,M,Addis Ababa,NaN,ETH,883,1783,2698,3628,...,4567,5519,6479,7362,298,7788,2,2,2,2015


In [16]:
# 원본데이터에서 컬럼을 필터링하고 최종 기록으로 정렬한 결과를  record 변수에 저장
record = pd.DataFrame(marathon_2015_2017, \
columns=['5K','10K','15K','20K','Half','25K','30K','35K','40K','Official Time'])\
.sort_values(by=['Official Time'])

# 위 저장된데이터를 2차원 리스트로 저장  변수명 record_list
record_list = record.values.tolist()
# record_list

In [17]:
# 각 그래프의 X 좌표로 사용할 데이터를 리스트로 만들어서 준비
xData = [5, 10, 15, 20, 21.098, 25, 30, 35, 40, 42.195 ]

In [18]:
# 왼쪽 차트
fig = Figure( figsize=(6,4), dpi=100)
ax = fig.add_subplot(111)
t_xdata, t_ydata, ml_xdata, ml_ydata, p_xdata, p_ydata = [], [], [], [], [], []
# t_xdata , t_ydata : 선수기록 버튼 클릭하면 나타날 차트의 좌표값
# (화면에 표시된 순위에 해당하는 선수의 각 구간별 기록)
# ml_xdata , ml_ydata : 회귀선이 그려질 데이터들이 저장
# (최종 회귀선을 찾은후 그려질 머신러닝 차트의 좌표값)
# p_xdata , p_ydata : 35, 40, 42.195 구간의 예측기록이 저장 
# ( 머신러닝 차트 좌표값중 35, 40, 42.195 의 예측의 예측기록을 별도 저장)

In [19]:
# x 좌표의 눈금
ax.set_xlim(0, 45)
# y 좌표의 눈금
ax.set_ylim(0, 13000)
# 축 라벨
ax.set_xlabel('Distance(km)')
ax.set_ylabel('Time(Second)')
# 차트 제목
ax.set_title('Records of runner')

# 애니메이션 구현을 위한 제어변수 생성 및 차트 모양 설정양 설정
ln, = ax.plot([], [], linestyle=':')  # 점선의 라인 설정 - 회귀선용
dn, = ax.plot([], [], 'ro')  # 빨간색 점 설정  - 최초 선수 기록 용
pn, = ax.plot([], [], 'bs')  # 파한색 스퀘어(네모) -  예측기록 용

In [20]:
# 오른쪽 그래프
# 경사하강법에 의한 코스트값이 매 100번째 step 마다 빨간 점으로 표시
grad_fig = Figure(figsize=(6,4), dpi=100)
grad_ax = grad_fig.add_subplot(111)
grad_ax.set_xlim(0, 5000)
grad_ax.set_ylim(0, 50000)
grad_ax.set_title('Cost Gradient Decent')
grad_ax.set_ylabel("Total Cost")
grad_ax.set_xlabel("Number of Traning")
# 100 번째 스텝마다 계산하여 차트에 표시할 코스트 값의 좌표를 저장할 리스트 생성 
# : x는 스탭값, y는 코스트값
g_xdata, g_ydata = [], []  
# 빨간색 점 - 애니메이션 구현을 위한 제어변수 생성 및 차트 모양 설정
gn, = grad_ax.plot([], [], 'ro')  

In [21]:
def seconds_to_hhmmss(seconds):
    hours = seconds // (60*60)   # 시 계산
    seconds %= (60*60)  # 시 를 뺀 나머지 초
    minutes = seconds // 60   # 분계산
    seconds %= 60   # 분을뺀 나머지 초
    return "%02i:%02i:%02i" % (hours, minutes, seconds)

In [22]:
def init():
    t_a = int(t_aSpbox.get()) -1
    ax.set_title('Records of runner #' + str(t_a + 1))
    ax.set_xlim(0, 45)
    ax.set_ylim(0, 13000)
    return dn,

In [23]:
def animateFrame(frames):
    # 스핀박스에 들어 있는 순위를 가져오되 -1 연산후 가져옵니다.
    # 1등이 0번에 있기때문에 -1 연산이 필요
    t_a = int(t_aSpbox.get()) -1
    
    # t_x 변수에  xData 에 있는 5,10,15,20.... 의 데이터 중 현재 프레임값에 
    # 해당하는 값을 저장합니다.(0~9)
    t_x = xData[int(frames)]  # [5, 10, 15, 20, 21.098, 25, 30, 35, 40, 42.195 ]
    
    # record_list  에 있는 값중 t_a 등수를 행으로, frames 값을 열로 하여 추출
    t_y = record_list[t_a][int(frames)]
    
    # 거리와 기록이 한쌍이 된데이터가 t_xdata, t_ydata 에 append
    # 왼쪽 차트에 빨간점이 찍힐 좌표들
    t_xdata.append(t_x)
    t_ydata.append(t_y)
    
    # dn 을 이용한 빨간 점 세트
    dn.set_data(t_xdata, t_ydata)
    # 현재의 x 좌표(거리구간), y좌표(기록(초)) 지점에 기록(초)를 시분초로 변환하여
    # annotate 로 차트에 추가 세트.
    ax.annotate( seconds_to_hhmmss(t_y) , (t_x, t_y), fontsize=8 )
    # 세트된 내용 차트에 표시
    fig.canvas.draw()
    return dn,

In [24]:
def update():
    # t_xdata 와 t_ydata  를 비웁니다
    t_xdata.clear()
    t_ydata.clear()
    # xData 갯수(10개) 만큼 프레임을 발생시켜서 애니메이트 실행하고 표시
    ani = FuncAnimation(fig, animateFrame,  frames=np.linspace(0,len(xData)-1, \
                                                      len(xData) ), init_func=init)
    # frames=np.linspace(0, len(xData)-1, len(xData) ) : 0~9 , 총 10개총 10개
    fig.canvas.draw()    

In [25]:
def draw_hypothesis(W, b):
    # 머신러닝 회귀선을 그리기 위한 리스트와 예측값을 표시하기 위한 리스트를 clear
    ml_xdata.clear()
    ml_ydata.clear()
    p_xdata.clear()
    p_ydata.clear()
    # xData 값을 10으로 나눠서 스케일링합니다. [5, 10, 15...] -> [0.5 , 1.0 , 1.5 ...]
    x_value = [ i/10 for i in xData]
    # 10 개의 x 데이터에 대해  y 값을 계산하여 저장합니다
    for x in range(10):
        h = W * x_value[x] + b   # 계산은 스케일링 한 데이터로 계산
        ml_xdata.append(xData[x])  # 좌표 저장은 스케일링 전 데이터로 저장
        ml_ydata.append(h)
    # 저장한 데이터 차트에 세트
    ln.set_data(ml_xdata, ml_ydata)

In [26]:
def learning():
    import tensorflow as tf
    import numpy as np
    
    t_a = int(t_aSpbox.get())   # 대상 데이터의 순위
    t_t = int(t_tSpbox.get())   # 트레이닝 횟수
    t_r = float(t_rSpbox.get()) # Running Rate 1회 학습 비율
    
    # x 축 데이터를 학습데이터와 테스트데이터로 분리
    # xData 에 있는 값들을 0~6 번까지의 데이터와이후 데이터로 분리 & 스케일링 실행
    # 스케일링 : 10으로 모두 나눠서 0~10사이의 값으로 변환
    x_train = [ i/10 for i in xData[0:7] ]
    # [5, 10, 15, 20, 21.098, 25, 30] ->  [0.5, 1.0, 1.5, 2.0, 2.1098, 2.5, 3.0]
    
    # y축값: 0~6 까지의 기록
    y_train = record_list[t_a-1][0:7]
    
    # Sequential : 텐서플로안에 포함된 케라스 모듈이 갖고 있는 인공지능 모델중 하나
    # Sequential 모델의 생성
    model =  tf.keras.models.Sequential()
    # Sequential 모델은 입력되는 데이터들의 모든 경우의 수를 활용한 패턴 서칭으로 
    # 최적의 예측값을 위한 학습을 진행합니다

    # 입력 단자 정의  (입력 : 1차원의 데이터 )
    model.add(tf.keras.layers.Dense(1, input_dim=1))
    # SGD : 경사하강법의 약자 - 아래 코드는 경사하강법을 적용할 객체를 생성
    sgd = tf.keras.optimizers.SGD(lr=t_r)  # lr:learning rate
    # 모델의 설계단계 . 성능평가 기준은 mse , 최적화 도구 : sgd
    model.compile(loss='mse', optimizer=sgd)
    # 학습
    history = model.fit(x_train, y_train, epochs=t_t, batch_size=128)
    # epochs : 경사하강 횟수, batch_size : 1회당 학습 용량 
    # 결과는 history 객체에 저장됩니다 (5000개의 코스트값 , 회귀 계수 등)
    
    # log_ScrolledText 에 써넣을 결과의 서두(순위, 학습횟수, 러닝레이트 등)
    log_ScrolledText.insert(END, '\n\n')
    log_ScrolledText.insert(END, "%10s %4i %10s %6i %20s %10.8f" % \
    ('Runner #',t_a,', No. of train is',t_t,',learing rate is ',t_r)+'\n', \
                            'TITLE')
    # 학습회차와 코스트값이 표시될 맨위의 열제목들
    log_ScrolledText.insert(END, '\n\nCost Decent\n\n','HEADER')
    log_ScrolledText.insert(END, "%20s %20s" % ('Step', 'Cost')+'\n\n')
    
    # 트레이닝 횟수만큼 반복 실행하되 100 회마다 출력 및 리스트에 저장(회차, 코스트값)
    for step in range(t_t):
        # 100번마다 실행  각 스탭값과 코스트 값을 출력 및 리스트에 저장
        if step % 100 == 0:
            # loss == cost  현재스텝의 코스값 저장
            cost_val = history.history['loss'][step]
            # 그리고 그 100회마다의 스탭값과 코스값을 그래프의 좌표값으로 append
            g_xdata.append(step)
            g_ydata.append(cost_val)
            # 현재 스탭값과 코스트값을 화면에 출력
            log_ScrolledText.insert(END, "%20i %20.5f" % (step, cost_val)+'\n')
            
    grad_ax.plot(g_xdata, g_ydata, 'ro')
    grad_ax.set_title('The minimum cost is %10.3f at %5i times' % (cost_val, step))
    grad_fig.canvas.draw()  
    
    # y = x1w1 + x2w2 + x3w3 + ....+ b
    # 회귀 계수값 추출하여 별도의 변수에 저장여 별도의 변수에 저장
    W_val = model.layers[0].get_weights()[0][0]
    b_val  = model.layers[0].get_weights()[1]
    
    # 얻어낸 최적의 회귀 계수들을 화면에 출력합니다
    log_ScrolledText.insert(END, "%20s" % ('\n\nHypothesis=X*W+b\n\n'), 'HEADER')
    log_ScrolledText.insert(END, "W_val : %.4f , b_val : %.4f\n\n" % (W_val, b_val), 
                                                                            'HEADER')
    # 얻어낸 최적의 회귀계수들을 이용하여 회귀선을 차트에 그리는 함수
    draw_hypothesis(W_val, b_val)
    
    # 35, 40, 42.195 데이터를 입력하여 예측(predict) 합니다.
    # Testing our model - 예측값 출력을 위한 제목들
    log_ScrolledText.insert(END, "%20s" % ('\n\nRecords Prediction\n\n'), 'HEADER')
    log_ScrolledText.insert(END, "%20s %20s %20s %20s" % ('Distance(km)', \
                       'Real record', 'ML Prediction', 'Variation(Second)')+'\n\n')
    
    #7번부터 9번꺄지의 숫자로 반복 실행
    for index in range(7, 10):
        # 예측을 위한 거리 35, 40, 42.195 를 3.5  4.0  4.2195 로 변경(스케일링)
        x_value = xData[index] / 10
        # 예측값으로 차트에 표시할 내용의 x 좌표값으로 저장 append
        p_xdata.append(xData[index])
        # 변환된 x 데이터로 예측(하나씩 차례로)
        time = model.predict(np.array([x_value]))
        # 예측된 시간 데이터를 y좌표 리스트에 저장
        p_ydata.append(time[0][0])
        
        # 구간거리, 실제기록, 예측기록,  오차값 순으로 출력합니다  (index - 7,8,9)
        log_ScrolledText.insert(END, "%20.3f %20s %20s %20i" % ( xData[index], \
                seconds_to_hhmmss(t_ydata[index]), seconds_to_hhmmss(time[0][0]), \
                                               (t_ydata[index] - time[0][0]))+'\n')
        
    # 예측값 좌표들로 차트에 표시
    pn.set_data(p_xdata, p_ydata)
    fig.canvas.draw()

In [27]:
main = Tk() #main 윈도우 객체 생성
main.title("마라톤 기록") # 타이틀 표시
main.geometry() # 크기 및 위치 
# 윈도우 제목이 실린 타이틀 설정 및 표시
label=Label(main, text='머신러닝 마라톤 기록 예측')
label.config(font=("Courier", 18))
label.grid(row=0,column=0,columnspan=6)

# 왼쪽 차트에 표시할 기록의 순위를 입력받은 스핀 박스와 라벨와 라벨
t_aVal  = DoubleVar(value=1)
t_aSpbox = Spinbox(main, textvariable=t_aVal ,from_=0, to=len(record_list), \
                                                       increment=1, justify=RIGHT)
t_aSpbox.grid(row=1,column=1)
t_aLabel=Label(main, text='선수의 순위 : ')                
t_aLabel.grid(row=1,column=0)

# 머신러닝 트레이닝 횟수 입력 받을 스핀박스와 라벨
t_tVal  = DoubleVar(value=5000)
t_tSpbox = Spinbox(main, textvariable=t_tVal ,from_=0, to=100000, increment=500, \
                                                                       justify=RIGHT)
t_tSpbox.grid(row=1,column=3)
t_tLabel=Label(main, text='트레이닝 횟수 : ')                
t_tLabel.grid(row=1,column=2)

# 경사하강계산 간격
t_rVal  = DoubleVar(value=0.01)
t_rSpbox = Spinbox(main, textvariable=t_rVal ,from_=0, to=1, increment=0.001, \
                                                                       justify=RIGHT)
t_rSpbox.grid(row=1,column=5)
t_rLabel=Label(main, text='경사하강 간격 : ')                
t_rLabel.grid(row=1,column=4)

# 버튼들 - sticky : 동서남북 정렬 기준  W, E 왼쪽 오른쪽 양쪽정렬
btn1 = Button(main,text="선수 기록", height=2, command=lambda:update())
btn1.grid(row=2, column=0, columnspan=3, sticky=(W, E))
btn2 = Button(main,text="Machine Learing", height=2, command=lambda:learning())
btn2.grid(row=2, column=3, columnspan=3, sticky=(W, E))

# 윈도우에 표시될 차트를 위한 피겨캔버스 연결 및 위치 지정
canvas = FigureCanvasTkAgg(fig, main)
canvas.get_tk_widget().grid(row=3,column=0,columnspan=3) 
grad_canvas = FigureCanvasTkAgg(grad_fig, main)
grad_canvas.get_tk_widget().grid(row=3,column=3,columnspan=3)

# 스크롤드텍스 표시 : HTML5의 TEXTAREA 성격의 텍스트 상자
log_ScrolledText = tkst.ScrolledText(main, height=15)
log_ScrolledText.grid(row=4,column=0,columnspan=6, sticky=(N, S, W, E))
log_ScrolledText.configure(font='굴림')

# Html5의 스타일시트 내에 css 를 지정해두고 이름 불러 쓸수 있는 기능을 아래와 같이
# 구현 할수 있습니다
log_ScrolledText.tag_config('HEADER',foreground='gray',font=("굴림",14), underline=1)
log_ScrolledText.tag_config('TITLE', foreground='orange', font=("굴림", 18), \
                                                    underline=1, justify='center')
log_ScrolledText.tag_config('RESULT', foreground='blue', font=("굴림", 12))

In [28]:
main.mainloop()

Epoch 1/5000
1/1 [==============================] - 0s 187ms/step - loss: 20661810.0000
Epoch 2/5000
1/1 [==============================] - 0s 3ms/step - loss: 16939620.0000
Epoch 3/5000
1/1 [==============================] - 0s 3ms/step - loss: 13891096.0000
Epoch 4/5000
1/1 [==============================] - 0s 2ms/step - loss: 11394295.0000
Epoch 5/5000
1/1 [==============================] - 0s 3ms/step - loss: 9349348.0000
Epoch 6/5000
1/1 [==============================] - 0s 3ms/step - loss: 7674466.5000
Epoch 7/5000
1/1 [==============================] - 0s 3ms/step - loss: 6302663.5000
Epoch 8/5000
1/1 [==============================] - 0s 2ms/step - loss: 5179080.0000
Epoch 9/5000
1/1 [==============================] - 0s 2ms/step - loss: 4258784.5000
Epoch 10/5000
1/1 [==============================] - 0s 2ms/step - loss: 3504981.7500
Epoch 11/5000
1/1 [==============================] - 0s 3ms/step - loss: 2887534.2500
Epoch 12/5000
1/1 [==============================] - 0s 3

Epoch 98/5000
1/1 [==============================] - 0s 3ms/step - loss: 60344.5977
Epoch 99/5000
1/1 [==============================] - 0s 3ms/step - loss: 60022.3711
Epoch 100/5000
1/1 [==============================] - 0s 4ms/step - loss: 59701.8203
Epoch 101/5000
1/1 [==============================] - 0s 3ms/step - loss: 59383.1211
Epoch 102/5000
1/1 [==============================] - 0s 4ms/step - loss: 59066.0273
Epoch 103/5000
1/1 [==============================] - 0s 3ms/step - loss: 58750.6523
Epoch 104/5000
1/1 [==============================] - 0s 3ms/step - loss: 58437.0039
Epoch 105/5000
1/1 [==============================] - 0s 3ms/step - loss: 58125.0625
Epoch 106/5000
1/1 [==============================] - 0s 2ms/step - loss: 57814.8164
Epoch 107/5000
1/1 [==============================] - 0s 3ms/step - loss: 57506.1953
Epoch 108/5000
1/1 [==============================] - 0s 3ms/step - loss: 57199.2266
Epoch 109/5000
1/1 [==============================] - 0s 3ms/step -

1/1 [==============================] - 0s 2ms/step - loss: 36143.2617
Epoch 195/5000
1/1 [==============================] - 0s 2ms/step - loss: 35951.4688
Epoch 196/5000
1/1 [==============================] - 0s 2ms/step - loss: 35760.6641
Epoch 197/5000
1/1 [==============================] - 0s 2ms/step - loss: 35570.9023
Epoch 198/5000
1/1 [==============================] - 0s 3ms/step - loss: 35382.1602
Epoch 199/5000
1/1 [==============================] - 0s 3ms/step - loss: 35194.4492
Epoch 200/5000
1/1 [==============================] - 0s 2ms/step - loss: 35007.6836
Epoch 201/5000
1/1 [==============================] - 0s 3ms/step - loss: 34821.9805
Epoch 202/5000
1/1 [==============================] - 0s 3ms/step - loss: 34637.2930
Epoch 203/5000
1/1 [==============================] - 0s 2ms/step - loss: 34453.5703
Epoch 204/5000
1/1 [==============================] - 0s 3ms/step - loss: 34270.8516
Epoch 205/5000
1/1 [==============================] - 0s 3ms/step - loss: 34089.

1/1 [==============================] - 0s 3ms/step - loss: 21735.9355
Epoch 291/5000
1/1 [==============================] - 0s 3ms/step - loss: 21621.7207
Epoch 292/5000
1/1 [==============================] - 0s 3ms/step - loss: 21508.1543
Epoch 293/5000
1/1 [==============================] - 0s 3ms/step - loss: 21395.1855
Epoch 294/5000
1/1 [==============================] - 0s 3ms/step - loss: 21282.8223
Epoch 295/5000
1/1 [==============================] - 0s 3ms/step - loss: 21171.0508
Epoch 296/5000
1/1 [==============================] - 0s 3ms/step - loss: 21059.9102
Epoch 297/5000
1/1 [==============================] - 0s 3ms/step - loss: 20949.3535
Epoch 298/5000
1/1 [==============================] - 0s 3ms/step - loss: 20839.3867
Epoch 299/5000
1/1 [==============================] - 0s 2ms/step - loss: 20730.0137
Epoch 300/5000
1/1 [==============================] - 0s 3ms/step - loss: 20621.2363
Epoch 301/5000
1/1 [==============================] - 0s 2ms/step - loss: 20513.

1/1 [==============================] - 0s 2ms/step - loss: 13159.0713
Epoch 387/5000
1/1 [==============================] - 0s 2ms/step - loss: 13091.0908
Epoch 388/5000
1/1 [==============================] - 0s 3ms/step - loss: 13023.4795
Epoch 389/5000
1/1 [==============================] - 0s 3ms/step - loss: 12956.2207
Epoch 390/5000
1/1 [==============================] - 0s 3ms/step - loss: 12889.3330
Epoch 391/5000
1/1 [==============================] - 0s 3ms/step - loss: 12822.8232
Epoch 392/5000
1/1 [==============================] - 0s 2ms/step - loss: 12756.6240
Epoch 393/5000
1/1 [==============================] - 0s 3ms/step - loss: 12690.8154
Epoch 394/5000
1/1 [==============================] - 0s 3ms/step - loss: 12625.3574
Epoch 395/5000
1/1 [==============================] - 0s 3ms/step - loss: 12560.2354
Epoch 396/5000
1/1 [==============================] - 0s 2ms/step - loss: 12495.4922
Epoch 397/5000
1/1 [==============================] - 0s 3ms/step - loss: 12431.

...

1/1 [==============================] - 0s 2ms/step - loss: 542.1774
Epoch 4558/5000
1/1 [==============================] - 0s 3ms/step - loss: 542.1773
Epoch 4559/5000
1/1 [==============================] - 0s 3ms/step - loss: 542.1773
Epoch 4560/5000
1/1 [==============================] - 0s 3ms/step - loss: 542.1774
Epoch 4561/5000
1/1 [==============================] - 0s 2ms/step - loss: 542.1774
Epoch 4562/5000
1/1 [==============================] - 0s 2ms/step - loss: 542.1774
Epoch 4563/5000
1/1 [==============================] - 0s 3ms/step - loss: 542.1773
Epoch 4564/5000
1/1 [==============================] - 0s 3ms/step - loss: 542.1774
Epoch 4565/5000
1/1 [==============================] - 0s 2ms/step - loss: 542.1774
Epoch 4566/5000
1/1 [==============================] - 0s 2ms/step - loss: 542.1773
Epoch 4567/5000
1/1 [==============================] - 0s 2ms/step - loss: 542.1774
Epoch 4568/5000
1/1 [==============================] - 0s 3ms/step - loss: 542.1773
Epoch 45

1/1 [==============================] - 0s 3ms/step - loss: 542.1773
Epoch 4655/5000
1/1 [==============================] - 0s 3ms/step - loss: 542.1774
Epoch 4656/5000
1/1 [==============================] - 0s 2ms/step - loss: 542.1774
Epoch 4657/5000
1/1 [==============================] - 0s 2ms/step - loss: 542.1774
Epoch 4658/5000
1/1 [==============================] - 0s 3ms/step - loss: 542.1774
Epoch 4659/5000
1/1 [==============================] - 0s 2ms/step - loss: 542.1774
Epoch 4660/5000
1/1 [==============================] - 0s 3ms/step - loss: 542.1774
Epoch 4661/5000
1/1 [==============================] - 0s 3ms/step - loss: 542.1774
Epoch 4662/5000
1/1 [==============================] - 0s 3ms/step - loss: 542.1774
Epoch 4663/5000
1/1 [==============================] - 0s 3ms/step - loss: 542.1773
Epoch 4664/5000
1/1 [==============================] - 0s 3ms/step - loss: 542.1774
Epoch 4665/5000
1/1 [==============================] - 0s 2ms/step - loss: 542.1774
Epoch 46

1/1 [==============================] - 0s 3ms/step - loss: 542.1774
Epoch 4752/5000
1/1 [==============================] - 0s 3ms/step - loss: 542.1774
Epoch 4753/5000
1/1 [==============================] - 0s 3ms/step - loss: 542.1774
Epoch 4754/5000
1/1 [==============================] - 0s 3ms/step - loss: 542.1774
Epoch 4755/5000
1/1 [==============================] - 0s 3ms/step - loss: 542.1774
Epoch 4756/5000
1/1 [==============================] - 0s 2ms/step - loss: 542.1774
Epoch 4757/5000
1/1 [==============================] - 0s 3ms/step - loss: 542.1773
Epoch 4758/5000
1/1 [==============================] - 0s 4ms/step - loss: 542.1774
Epoch 4759/5000
1/1 [==============================] - 0s 3ms/step - loss: 542.1774
Epoch 4760/5000
1/1 [==============================] - 0s 3ms/step - loss: 542.1774
Epoch 4761/5000
1/1 [==============================] - 0s 3ms/step - loss: 542.1774
Epoch 4762/5000
1/1 [==============================] - 0s 2ms/step - loss: 542.1774
Epoch 47

1/1 [==============================] - 0s 3ms/step - loss: 542.1774
Epoch 4849/5000
1/1 [==============================] - 0s 3ms/step - loss: 542.1774
Epoch 4850/5000
1/1 [==============================] - 0s 3ms/step - loss: 542.1774
Epoch 4851/5000
1/1 [==============================] - 0s 3ms/step - loss: 542.1774
Epoch 4852/5000
1/1 [==============================] - 0s 2ms/step - loss: 542.1773
Epoch 4853/5000
1/1 [==============================] - 0s 3ms/step - loss: 542.1774
Epoch 4854/5000
1/1 [==============================] - 0s 2ms/step - loss: 542.1774
Epoch 4855/5000
1/1 [==============================] - 0s 3ms/step - loss: 542.1774
Epoch 4856/5000
1/1 [==============================] - 0s 3ms/step - loss: 542.1774
Epoch 4857/5000
1/1 [==============================] - 0s 3ms/step - loss: 542.1774
Epoch 4858/5000
1/1 [==============================] - 0s 3ms/step - loss: 542.1774
Epoch 4859/5000
1/1 [==============================] - 0s 2ms/step - loss: 542.1774
Epoch 48

1/1 [==============================] - 0s 2ms/step - loss: 542.1774
Epoch 4946/5000
1/1 [==============================] - 0s 3ms/step - loss: 542.1774
Epoch 4947/5000
1/1 [==============================] - 0s 2ms/step - loss: 542.1774
Epoch 4948/5000
1/1 [==============================] - 0s 2ms/step - loss: 542.1774
Epoch 4949/5000
1/1 [==============================] - 0s 3ms/step - loss: 542.1774
Epoch 4950/5000
1/1 [==============================] - 0s 4ms/step - loss: 542.1774
Epoch 4951/5000
1/1 [==============================] - 0s 3ms/step - loss: 542.1774
Epoch 4952/5000
1/1 [==============================] - 0s 4ms/step - loss: 542.1774
Epoch 4953/5000
1/1 [==============================] - 0s 3ms/step - loss: 542.1773
Epoch 4954/5000
1/1 [==============================] - 0s 2ms/step - loss: 542.1774
Epoch 4955/5000
1/1 [==============================] - 0s 2ms/step - loss: 542.1774
Epoch 4956/5000
1/1 [==============================] - 0s 3ms/step - loss: 542.1774
Epoch 49